In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px

import pandas as pd
import seaborn as sns

from manusim.metrics import Metrics

In [ ]:
experiments_folder = Path("C:/github/master/manufacturing-scheduling/data/experiments")
experiment_path = list(experiments_folder.rglob("*2508061632_sbUpdateTest_0*"))
experiment_path = experiment_path[0]
print(experiment_path )

run = 1

In [ ]:
dbr_metrics = Metrics(experiment_folder = experiment_path)
dbr_metrics.read_logs()

In [ ]:
df_products = dbr_metrics.products_logs.copy()
df_products["product"] = df_products["product"].apply(lambda x: x.replace("produto", "_p") if isinstance(x, str) else x)
df_products["variable"] = df_products["variable"] + df_products["product"]
df_products = df_products.drop("product", axis=1)
df_resources = dbr_metrics.resources_logs.copy()
df_resources["resource"] = df_resources["resource"].apply(lambda x: x.replace("recurso", "_r") if isinstance(x, str) else x)
df_resources["variable"] = df_resources["variable"] + df_resources["resource"]
df_resources = df_resources.drop("resource", axis=1)

df_general = pd.concat(
    [
        df_products,
        df_resources,
        dbr_metrics.general_logs
    ]
)
df_general


In [ ]:
products = dbr_metrics.products_logs["product"].unique()

df = dbr_metrics.products_logs
df = pd.concat([df, dbr_metrics.general_logs])

variables = [
    "shipping_buffer_target",
    "shipping_buffer_level",
    "released",
    "schedule_consumed",
    "wip_log",
    "fg_log",
    "schedule_planned",
    "constraint_buffer_level"
]


df = df.loc[df["run"]==run]
df

In [ ]:

save_plots = experiment_path / f"plots_{run}"
save_plots.mkdir(exist_ok=True, parents=True)

for product in products:
    
    df_tmp = df.loc[
        (df["variable"].isin(variables)) & 
        ((df["product"]==product) | (df["product"].isna()))
    ]
    fig = px.line(df_tmp, x="time", y="value", color="variable", markers=True, title=product)
    fig.write_html(save_plots / f"timeline_{product}.html")

In [ ]:
df_resouces = dbr_metrics.resources_logs
df_resouces = df_resouces.loc[df_resouces["run"]==run]
print(df_resouces["variable"].unique())

In [ ]:
df_break = df_resouces.loc[df_resouces["variable"]=="breakdowns"]
df_break.loc[:,"variable"] = df_break["variable"]+df_break["resource"]
df_tmp = df_break[["time", "value", "variable"]]
df_tmp.loc[:,"time"] = df_tmp["time"] + df_tmp["value"]
df_tmp.loc[:, "value"] = 0
df_break = pd.concat([df_break, df_tmp]).reset_index().sort_values("time")
df_break

In [ ]:
df_wip = df.loc[df["variable"]=="wip_log"]
df_wip["variable"] = df_wip["variable"] + "_" + df_wip["product"]
df_wip = pd.concat([df_wip, df_break])
fig = px.line(df_wip, x="time", y="value", color="variable", markers=True)
fig.write_html(save_plots / f"timeline_wip.html") 


In [ ]:
df.loc[
    (df["variable"]=="fg_log")
].sort_values(by=["value", "time"])

In [ ]:

df_queue = df_resouces.loc[df_resouces["variable"]=="queues"]
df_queue["variable"] = df_queue["variable"] + "_" + df_queue["resource"]
df_queue = pd.concat([df_queue, df_break])
fig = px.line(df_queue, x="time", y="value", color="variable", markers=True)
fig.write_html(save_plots / f"timeline_queue.html") 

In [ ]:
df_resouces = dbr_metrics.resources_logs
print(df_resouces["variable"].unique())


In [ ]:
df_break = df_resouces.loc[df_resouces["variable"]=="breakdowns"]
df_break.loc[:,"variable"] = df_break["variable"]+df_break["resource"]
df_tmp = df_break[["time", "value", "variable"]]
df_tmp.loc[:,"time"] = df_tmp["time"] + df_tmp["value"]
df_tmp.loc[:, "value"] = 0
df_break = pd.concat([df_break, df_tmp]).reset_index().sort_values("time")
df_break


In [ ]:
df = dbr_metrics.products_logs.copy()
df = df.loc[df["variable"]=="schedule_planned"]

fig = px.line(df, x="time", y="value", color="product", markers=True)
fig.write_html(save_plots / f"timeline_schedule_planned.html") 

In [ ]:
df = dbr_metrics.products_logs.copy()
df = df.loc[df["variable"]=="schedule_consumed"]

fig = px.line(df, x="time", y="value", color="product", markers=True)
fig.write_html(save_plots / f"timeline_schedule_consumed.html") 